# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator
import random

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [3]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [6]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output)
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [10]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [13]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [14]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [15]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.6168323775286386),
 (1, 1.6168323775286386),
 (2, 1.6168323775286386),
 (3, 1.6168323775286386),
 (4, 1.6168323775286386)]

## Inverted index

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('banana', 2), ('is', 18), ('it', 18), ('what', 10)]),
 (1, [('a', 2)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.10737639317598113),
   (None, 0.2016565884172279),
   (None, 0.2590328900453609),
   (None, 0.27763991383208164),
   (None, 0.29160019408562454),
   (None, 0.3147036925420177),
   (None, 0.34729302103302095),
   (None, 0.35172372437553723),
   (None, 0.3846875807171759),
   (None, 0.4430081046537878),
   (None, 0.45234893737574333)]),
 (1,
  [(None, 0.5011976133567866),
   (None, 0.5074786651514601),
   (None, 0.656504851323703),
   (None, 0.6585117090174429),
   (None, 0.6603983376408654),
   (None, 0.7291440653008885),
   (None, 0.7682417826830741),
   (None, 0.7994772526622566),
   (None, 0.8250907754936323),
   (None, 0.8398900004105115),
   (None, 0.8441087907984488),
   (None, 0.856037340760352),
   (None, 0.8698692994263838),
   (None, 0.9159053298661012),
   (None, 0.9162705567185115),
   (None, 0.9278885274263078),
   (None, 0.9341893877002104),
   (None, 0.9890437495557943),
   (None, 0.9970025339561236)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [21]:
import random

def MAP(num_list):
    return max(num_list)


def REDUCE(num_list):
    return max(num_list)


def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]


record = RECORDREADER(100)
print(record)

part_count = 5
record_partitional = [record[d:d + part_count] for d in range(0, len(record), part_count)]
print(record_partitional)

print(REDUCE(list(map(lambda x: MAP(x), record_partitional))))

[62, 16, 12, 35, 35, 44, 6, 39, 15, 12, 97, 10, 5, 57, 93, 32, 39, 100, 79, 83, 35, 53, 84, 95, 50, 0, 77, 93, 7, 29, 49, 97, 63, 2, 4, 61, 18, 94, 94, 93, 69, 56, 13, 63, 98, 22, 32, 15, 2, 76, 78, 38, 36, 31, 41, 37, 4, 88, 66, 92, 84, 81, 22, 95, 81, 14, 70, 70, 72, 89, 76, 40, 91, 21, 23, 39, 52, 19, 99, 14, 90, 2, 20, 53, 31, 43, 94, 11, 34, 22, 20, 53, 35, 66, 57, 15, 41, 67, 20, 21]
[[62, 16, 12, 35, 35], [44, 6, 39, 15, 12], [97, 10, 5, 57, 93], [32, 39, 100, 79, 83], [35, 53, 84, 95, 50], [0, 77, 93, 7, 29], [49, 97, 63, 2, 4], [61, 18, 94, 94, 93], [69, 56, 13, 63, 98], [22, 32, 15, 2, 76], [78, 38, 36, 31, 41], [37, 4, 88, 66, 92], [84, 81, 22, 95, 81], [14, 70, 70, 72, 89], [76, 40, 91, 21, 23], [39, 52, 19, 99, 14], [90, 2, 20, 53, 31], [43, 94, 11, 34, 22], [20, 53, 35, 66, 57], [15, 41, 67, 20, 21]]
100


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [22]:
def MAP(num):
    return (1, num)


def REDUCE(key, nums: Iterator[NamedTuple]):
    sum = 0
    count = 0
    for n in nums:
        sum += n
        count += 1
    yield sum / count


def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]


map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(flatten(map(lambda x: REDUCE(*x), shuffle_output)))
print(reduce_output)

[(1, 89), (1, 79), (1, 30), (1, 60), (1, 56), (1, 69), (1, 19), (1, 95), (1, 51), (1, 47), (1, 89), (1, 37), (1, 2), (1, 75), (1, 89), (1, 25), (1, 68), (1, 5), (1, 86), (1, 36), (1, 94), (1, 90), (1, 85), (1, 45), (1, 74), (1, 76), (1, 60), (1, 93), (1, 81), (1, 28), (1, 47), (1, 90), (1, 29), (1, 45), (1, 37), (1, 16), (1, 5), (1, 81), (1, 89), (1, 38), (1, 79), (1, 63), (1, 53), (1, 93), (1, 41), (1, 88), (1, 56), (1, 33), (1, 68), (1, 59), (1, 45), (1, 7), (1, 58), (1, 93), (1, 56), (1, 81), (1, 63), (1, 44), (1, 3), (1, 13), (1, 11), (1, 90), (1, 77), (1, 56), (1, 93), (1, 29), (1, 70), (1, 56), (1, 75), (1, 31), (1, 33), (1, 98), (1, 56), (1, 88), (1, 88), (1, 86), (1, 16), (1, 66), (1, 97), (1, 91), (1, 13), (1, 47), (1, 20), (1, 63), (1, 52), (1, 43), (1, 48), (1, 34), (1, 32), (1, 97), (1, 65), (1, 81), (1, 7), (1, 71), (1, 53), (1, 100), (1, 53), (1, 32), (1, 90), (1, 70)]
[(1, [89, 79, 30, 60, 56, 69, 19, 95, 51, 47, 89, 37, 2, 75, 89, 25, 68, 5, 86, 36, 94, 90, 85, 45, 74, 

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [23]:
def group_by_key(iterable):
    sorted_data = sorted(iterable, key=lambda x: x[0])
    grouped_data = {}
    for key, value in sorted_data:
        if key not in grouped_data:
            grouped_data[key] = [value]
        else:
            grouped_data[key].append(value)
    result = []
    for key, value in grouped_data.items():
        result.append((key, value))
    return result

def MAP(num):
    return (1, num)


def REDUCE(key, nums: Iterator[NamedTuple]):
    sum = 0
    count = 0
    for n in nums:
        sum += n
        count += 1
    yield sum / count


def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(flatten(map(lambda x: REDUCE(*x), shuffle_output)))
print(reduce_output)

[(1, 36), (1, 64), (1, 22), (1, 69), (1, 58), (1, 38), (1, 48), (1, 60), (1, 4), (1, 79), (1, 53), (1, 72), (1, 75), (1, 55), (1, 79), (1, 46), (1, 91), (1, 22), (1, 58), (1, 3), (1, 40), (1, 82), (1, 10), (1, 34), (1, 82), (1, 17), (1, 86), (1, 24), (1, 11), (1, 87), (1, 32), (1, 86), (1, 12), (1, 31), (1, 68), (1, 68), (1, 26), (1, 30), (1, 50), (1, 78), (1, 55), (1, 10), (1, 40), (1, 45), (1, 22), (1, 48), (1, 60), (1, 74), (1, 75), (1, 4), (1, 32), (1, 70), (1, 62), (1, 68), (1, 11), (1, 42), (1, 23), (1, 43), (1, 17), (1, 8), (1, 97), (1, 2), (1, 38), (1, 18), (1, 4), (1, 62), (1, 6), (1, 67), (1, 87), (1, 85), (1, 63), (1, 12), (1, 10), (1, 52), (1, 49), (1, 92), (1, 57), (1, 59), (1, 15), (1, 70), (1, 8), (1, 6), (1, 100), (1, 84), (1, 65), (1, 44), (1, 51), (1, 24), (1, 67), (1, 68), (1, 98), (1, 64), (1, 91), (1, 49), (1, 26), (1, 86), (1, 88), (1, 89), (1, 73), (1, 18)]
[(1, [36, 64, 22, 69, 58, 38, 48, 60, 4, 79, 53, 72, 75, 55, 79, 46, 91, 22, 58, 3, 40, 82, 10, 34, 82, 17,

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [24]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs


# ===================================================================================================================================================================================

from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, word)

def REDUCE(key:str, value:Iterator[str]):
  yield key

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, ['', 'banana', 'is', 'it', 'what']), (1, ['a'])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [25]:
from collections import defaultdict

def MAP(el_list):
    mapped_result = defaultdict(list)
    for t in el_list:
        if C(t):  # Предикат C
            mapped_result[t].append(t)
    return mapped_result.items()


def REDUCE(mapped_items):
    reduced_result = []
    print(mapped_items)
    for values in mapped_items:
        for value in values:
            reduced_result.append(value[0])
    return reduced_result

def C(t):
    return t[0] % 2 == 0  # Выбираем только кортежи с четным первым элементом


def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

record = RECORDREADER(100)
print(record)

part_count = 5
record_partitional = [record[d:d + part_count] for d in range(0, len(record), part_count)]
print(record_partitional)

print(REDUCE(list(map(lambda x: MAP(x), record_partitional))))

[(20, 12, 21), (39, 9, 76), (78, 65, 21), (98, 38, 22), (16, 48, 42), (6, 86, 96), (43, 49, 63), (48, 0, 78), (42, 49, 72), (71, 92, 29), (33, 54, 2), (47, 36, 94), (53, 88, 50), (77, 62, 96), (81, 84, 83), (52, 50, 98), (39, 40, 57), (47, 71, 49), (8, 20, 65), (64, 100, 67), (99, 22, 95), (29, 74, 37), (26, 23, 78), (54, 99, 89), (30, 6, 58), (20, 40, 85), (21, 68, 92), (74, 51, 25), (19, 87, 95), (86, 71, 77), (21, 73, 64), (11, 100, 31), (48, 81, 19), (1, 13, 64), (87, 12, 74), (82, 13, 15), (14, 84, 13), (91, 31, 92), (72, 2, 3), (35, 79, 91), (16, 84, 87), (45, 54, 64), (99, 68, 54), (98, 52, 43), (25, 4, 26), (73, 42, 28), (70, 83, 70), (48, 87, 72), (18, 21, 74), (13, 67, 40), (45, 16, 45), (65, 50, 8), (56, 73, 24), (14, 34, 25), (23, 85, 34), (58, 5, 60), (54, 86, 24), (98, 49, 20), (0, 29, 54), (50, 93, 0), (67, 36, 17), (54, 58, 29), (96, 41, 44), (91, 20, 76), (75, 82, 9), (44, 59, 80), (34, 94, 97), (27, 99, 93), (59, 54, 56), (32, 16, 21), (88, 67, 0), (19, 35, 95), (57, 

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [26]:
S = set([1 ,12, 56])

def MAP(t):
  res_list = []
  for el in t:
    if el in S:
      res_list.append(el)
    res = tuple(res_list)
    return (res, res)


def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))
print(reduce_output)

[(1, 55, 74), (97, 74, 27), (93, 13, 16), (63, 96, 19), (48, 87, 44), (100, 50, 92), (94, 73, 87), (43, 40, 63), (32, 64, 26), (72, 2, 36), (75, 100, 12), (86, 57, 56), (0, 2, 92), (45, 76, 95), (27, 20, 3), (77, 83, 99), (39, 91, 67), (41, 29, 100), (54, 38, 90), (54, 68, 75), (35, 75, 60), (78, 53, 79), (82, 0, 55), (67, 33, 97), (81, 20, 7), (93, 52, 70), (89, 61, 13), (85, 26, 53), (47, 42, 23), (20, 79, 68), (14, 52, 6), (70, 10, 21), (76, 37, 51), (63, 15, 74), (74, 90, 57), (16, 83, 44), (52, 9, 17), (23, 54, 79), (10, 68, 34), (70, 80, 42), (35, 33, 59), (20, 100, 84), (50, 93, 13), (57, 28, 93), (90, 20, 38), (16, 68, 72), (36, 75, 45), (55, 1, 75), (9, 100, 21), (32, 27, 17), (31, 13, 17), (25, 10, 82), (38, 21, 52), (13, 77, 39), (90, 13, 85), (17, 97, 93), (45, 30, 7), (11, 53, 48), (79, 89, 84), (61, 98, 34), (37, 62, 69), (74, 62, 30), (75, 13, 92), (47, 62, 95), (41, 81, 94), (30, 25, 8), (68, 71, 87), (38, 16, 99), (33, 15, 17), (100, 17, 82), (25, 58, 1), (50, 75, 70),

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [27]:
def MAP(t):
  return (t, t)


def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))
print(reduce_output)

[(83, 40, 70), (94, 56, 14), (89, 61, 96), (92, 42, 27), (50, 75, 99), (36, 23, 99), (19, 92, 66), (36, 71, 98), (98, 91, 1), (79, 51, 52), (62, 7, 81), (36, 82, 94), (52, 90, 16), (84, 10, 18), (4, 60, 59), (77, 20, 73), (50, 34, 100), (70, 79, 66), (3, 58, 40), (87, 12, 25), (86, 17, 54), (93, 7, 2), (54, 14, 53), (30, 98, 5), (13, 75, 26), (8, 73, 39), (33, 90, 41), (69, 72, 49), (78, 80, 32), (9, 41, 77), (15, 37, 68), (42, 11, 3), (52, 34, 93), (20, 0, 22), (22, 2, 34), (55, 23, 21), (65, 2, 46), (86, 25, 6), (89, 18, 21), (46, 31, 27), (11, 60, 8), (80, 73, 41), (6, 77, 31), (43, 8, 89), (72, 75, 95), (94, 22, 53), (12, 39, 59), (18, 27, 65), (62, 10, 88), (39, 83, 83), (14, 96, 85), (96, 81, 47), (65, 46, 40), (97, 55, 82), (32, 19, 25), (18, 57, 48), (86, 40, 34), (90, 40, 71), (82, 11, 19), (18, 4, 41), (18, 41, 55), (26, 48, 18), (65, 77, 59), (95, 53, 61), (18, 28, 92), (56, 27, 91), (72, 23, 79), (87, 6, 34), (85, 74, 58), (62, 78, 70), (51, 34, 30), (82, 83, 78), (89, 18, 

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [28]:
def MAP(t):
  return (t, t)


def REDUCE(key, values: Iterator[NamedTuple]):
  if len(values) == 2:
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 3), random.randint(0, 3)) for i in range(count)]

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]
print(reduce_output)

[(2, 1), (3, 0), (0, 3), (0, 1), (0, 1), (0, 1), (2, 1), (1, 2), (1, 3), (2, 0), (0, 1), (2, 3), (0, 2), (1, 2), (3, 2), (2, 1), (3, 2), (1, 2), (1, 0), (3, 1), (2, 3), (1, 2), (3, 2), (3, 1), (0, 1), (0, 3), (1, 3), (3, 1), (3, 0), (3, 2), (0, 2), (1, 2), (0, 2), (2, 0), (0, 3), (2, 0), (3, 0), (2, 3), (0, 3), (1, 3), (0, 2), (1, 1), (3, 3), (1, 0), (0, 0), (0, 0), (0, 2), (2, 1), (1, 1), (0, 2), (0, 2), (3, 0), (1, 2), (3, 2), (3, 1), (2, 2), (1, 1), (3, 1), (1, 1), (2, 0), (1, 3), (2, 3), (1, 0), (2, 0), (2, 1), (1, 0), (1, 2), (1, 2), (0, 1), (2, 2), (2, 1), (1, 3), (1, 0), (2, 1), (0, 1), (3, 1), (0, 2), (2, 3), (3, 1), (0, 3), (3, 1), (2, 3), (3, 1), (1, 0), (0, 1), (0, 3), (0, 0), (0, 3), (0, 2), (3, 0), (1, 2), (3, 3), (3, 0), (3, 2), (2, 2), (1, 3), (2, 0), (2, 1), (3, 0), (0, 1)]
[((2, 1), (2, 1)), ((0, 2), (0, 2)), ((3, 3), (3, 3)), ((0, 2), (0, 2)), ((2, 3), (2, 3)), ((0, 2), (0, 2)), ((1, 3), (1, 3)), ((3, 2), (3, 2)), ((0, 3), (0, 3)), ((0, 3), (0, 3)), ((3, 2), (3, 2)), 

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [29]:
rels = [1, 2]
class Tuple: # класс кортежа, который хранит данные и имя представления из массива rels выше
  def __init__(self, data: tuple, rel_id: int):
    self.data = data
    self.rel_id = rel_id

def get_random_tuple(count):
  data = tuple([(random.randint(0, 3), random.randint(0, 3)) for i in range(count)])
  rel_id = rels[random.randint(0, len(rels) - 1)]
  return Tuple(data, rel_id)


def RECORDREADER(count):
    return [get_random_tuple(3) for i in range(count)]

def MAP(t: Tuple):
  return (t.data, t.rel_id)

def REDUCE(key, values: Iterator[NamedTuple]):
  if values == [rels[0]]: # если соответствующее значение является списком  [R] , создайте пару  (t,t)
    return (key, key)

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]
print(reduce_output)

[<__main__.Tuple object at 0x000002898DAE6220>, <__main__.Tuple object at 0x000002898C6EE580>, <__main__.Tuple object at 0x000002898DAEA6A0>, <__main__.Tuple object at 0x000002898DAEA280>, <__main__.Tuple object at 0x000002898DAEA3D0>, <__main__.Tuple object at 0x000002898DAEA310>, <__main__.Tuple object at 0x000002898DAEA760>, <__main__.Tuple object at 0x000002898DAEAA30>, <__main__.Tuple object at 0x000002898DAEA490>, <__main__.Tuple object at 0x000002898DAEA3A0>, <__main__.Tuple object at 0x000002898DAEABE0>, <__main__.Tuple object at 0x000002898DAEA9D0>, <__main__.Tuple object at 0x000002898DAEAF40>, <__main__.Tuple object at 0x000002898DAEAC40>, <__main__.Tuple object at 0x000002898DAEAB50>, <__main__.Tuple object at 0x000002898DAEA8E0>, <__main__.Tuple object at 0x000002898DAEA910>, <__main__.Tuple object at 0x000002898DAEA1C0>, <__main__.Tuple object at 0x000002898DAEAB80>, <__main__.Tuple object at 0x000002898DAEABB0>, <__main__.Tuple object at 0x000002898DAEA970>, <__main__.Tu

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [30]:
rels = [1, 2]
class Tuple: # класс кортежа, который хранит данные и имя представления из массива rels выше
  def __init__(self, data: tuple, rel_id: int):
    self.data = data
    self.rel_id = rel_id

def get_random_tuple():
  data = (random.randint(0, 3), random.randint(0, 3))
  rel_id = rels[random.randint(0, len(rels) - 1)]
  return Tuple(data, rel_id)


def RECORDREADER(count):
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):
  if t.rel_id == rels[0]:
    return (t.data[1], (t.rel_id, t.data[0]))
  else:
    return (t.data[0], (t.rel_id, t.data[1]))

def REDUCE(key, values: Iterator[NamedTuple]):
  res = []
  for v in values:
    res.append((v[0], key, v[1]))
  return res

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))
print(reduce_output)

[<__main__.Tuple object at 0x000002898DAE6AF0>, <__main__.Tuple object at 0x000002898DAE62E0>, <__main__.Tuple object at 0x000002898DAE6880>, <__main__.Tuple object at 0x000002898DAE6340>, <__main__.Tuple object at 0x000002898DAE6B80>, <__main__.Tuple object at 0x000002898DAE6550>, <__main__.Tuple object at 0x000002898DABF940>, <__main__.Tuple object at 0x000002898DABFC40>, <__main__.Tuple object at 0x000002898DABF550>, <__main__.Tuple object at 0x000002898DABFD90>, <__main__.Tuple object at 0x000002898DABF520>, <__main__.Tuple object at 0x000002898DABF250>, <__main__.Tuple object at 0x000002898DABF580>, <__main__.Tuple object at 0x000002898DABF7C0>, <__main__.Tuple object at 0x000002898C6C9730>, <__main__.Tuple object at 0x000002898C6C9280>, <__main__.Tuple object at 0x000002898C6C9760>, <__main__.Tuple object at 0x000002898C6C9F10>, <__main__.Tuple object at 0x000002898C6C9490>, <__main__.Tuple object at 0x000002898C6C9970>, <__main__.Tuple object at 0x000002898C6C9EE0>, <__main__.Tu

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [31]:
def get_random_tuple():
  return (random.randint(0, 3), random.randint(0, 3), random.randint(0, 3))

def RECORDREADER(count):
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):
  return (t[0], t[1])

def tetta(values):
  return sum(values)


def REDUCE(key, values: Iterator[NamedTuple]):
  x = tetta(values)
  return (key, x)

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))
print(reduce_output)

[(0, 0, 0), (3, 0, 2), (0, 1, 1), (1, 1, 1), (2, 0, 0), (3, 2, 1), (1, 3, 1), (2, 2, 1), (0, 2, 1), (2, 2, 0), (0, 1, 2), (0, 2, 2), (2, 3, 0), (0, 1, 3), (3, 1, 2), (0, 2, 3), (2, 1, 1), (0, 3, 1), (2, 1, 1), (3, 2, 2), (3, 1, 2), (1, 0, 0), (3, 1, 0), (1, 1, 0), (0, 1, 2), (0, 0, 1), (0, 0, 3), (2, 0, 3), (2, 3, 1), (3, 0, 0), (1, 2, 3), (0, 1, 2), (0, 0, 3), (2, 2, 2), (1, 1, 1), (3, 0, 0), (2, 0, 0), (1, 0, 1), (1, 1, 0), (2, 0, 0), (3, 1, 0), (3, 3, 2), (0, 3, 0), (1, 2, 2), (0, 0, 2), (3, 3, 2), (3, 0, 1), (0, 0, 3), (0, 3, 2), (2, 2, 1), (3, 2, 3), (3, 3, 1), (3, 1, 2), (2, 0, 3), (2, 3, 1), (1, 0, 2), (3, 1, 3), (2, 2, 2), (3, 3, 2), (1, 1, 2), (3, 1, 1), (1, 2, 1), (1, 2, 2), (3, 2, 1), (1, 3, 2), (0, 1, 1), (1, 2, 2), (3, 1, 3), (3, 2, 1), (2, 1, 1), (3, 3, 0), (3, 1, 3), (2, 0, 1), (2, 2, 1), (2, 3, 1), (0, 2, 3), (2, 2, 3), (2, 1, 3), (2, 2, 1), (0, 1, 1), (2, 1, 1), (1, 1, 0), (1, 0, 1), (0, 0, 0), (2, 1, 0), (1, 0, 0), (2, 2, 1), (2, 0, 0), (3, 3, 0), (0, 0, 0), (3, 3, 1)

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [32]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [33]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j, k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(small_mat.shape[0]):
    yield ((i, k), w * small_mat[i][j])

def REDUCE(key, values):
  (i, k) = key
  el_value = 0
  for v in values:
    el_value += v
  yield ((i, k), el_value)

Проверьте своё решение

In [34]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [35]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [36]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def RECORDREADER():
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      yield ((0, i, j), small_mat[i,j]) # первая матрица

  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((1, j, k), big_mat[j,k]) # вторая матрица

def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))


def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def MAP_MUL(k1, v1):
  (i, k) = k1
  yield ((i, k), v1)


def REDUCE_MUL(key, values):
  res_el_value = 0
  for v in values:
    res_el_value += v
  yield (key, res_el_value)


def GET_JOINED():
  for j in joined:
    yield j

joined = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
solution = MapReduce(GET_JOINED, MAP_MUL, REDUCE_MUL)
np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [37]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

# ===================================================================================================================================================================================

import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j])) # первая матрица
  yield first_mat

  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k])) # вторая матрица
  yield second_mat


def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))


def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]


def MAP_MUL(k1, v1):
  yield (k1, v1)


def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 4
reducers = 2
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)
np.allclose(reference_solution, asmatrix(solution)) # should return true


126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.3581488866157133), ((0, 1), 0.006955723686467427), ((0, 2), 0.46627276834143), ((0, 3), 0.056672035940517285), ((0, 4), 0.5068065045697484), ((0, 5), 0.3023896462291538), ((0, 6), 0.20103988610451712), ((0, 7), 0.213487269704608), ((0, 8), 0.28191232526846305), ((0, 9), 0.23431987590834358), ((0, 10), 0.6804018097167016), ((0, 11), 0.647756908669833), ((0, 12), 0.11129116722671283), ((0, 13), 0.33306953226870906), ((0, 14), 0.5531685964546295), ((0, 15), 0.27328840527958215), ((0, 16), 0.7087753947159547), ((0, 17), 0.07744105889385752), ((0, 18), 0.2297789139001805), ((0, 19), 0.2959601566662509), ((0, 20), 0.32441451794622617), ((0, 21), 0.2849325848861935), ((0, 22), 0.01295694194049164), ((0, 23), 0.4708813103046157), ((0, 24), 0.0737225604591652), ((0, 25), 0.20012703743255802), ((0, 26), 0.2587700764762574), ((0, 27), 0.11896585246423655), ((0, 28), 0.6330625840585138), ((0, 29), 0.25222676819869216), ((0, 30), 0.5406

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [38]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) 
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

# ===================================================================================================================================================================================

import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j])) 

  global maps
  split_size =  int(np.ceil(len(first_mat)/maps))
  for i in range(0, len(first_mat), split_size):
    yield first_mat[i:i+split_size]

  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k])) 

  split_size =  int(np.ceil(len(second_mat)/maps))
  for i in range(0, len(second_mat), split_size):
    yield second_mat[i:i+split_size]


def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))


def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]


def MAP_MUL(k1, v1):
  yield (k1, v1)


def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 3
reducers = 2
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)
np.allclose(reference_solution, asmatrix(solution))


126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.17105144766864955), ((0, 1), 0.3116890896489657), ((0, 2), 0.5139501002325411), ((0, 3), 0.013645811346658512), ((0, 4), 0.46285917299169926), ((0, 5), 0.32486827981794875), ((0, 6), 0.43301530832038365), ((0, 7), 0.35938293735251664), ((0, 8), 0.4873530219623644), ((0, 9), 0.3588793849391749), ((0, 10), 0.3096162310733503), ((0, 11), 0.6083782206619256), ((0, 12), 0.10917889842454966), ((0, 13), 0.5873488163985036), ((0, 14), 0.4062378563131502), ((0, 15), 0.1339650355968498), ((0, 16), 0.13510841294525572), ((0, 17), 0.18142870438193692), ((0, 18), 0.6323694670794139), ((0, 19), 0.3729837495467902), ((0, 20), 0.5343545280692334), ((0, 21), 0.41174674834702946), ((0, 22), 0.250523860453854), ((0, 23), 0.1887318239279611), ((0, 24), 0.48000059029661307), ((0, 25), 0.3025803614905155), ((0, 26), 0.4335439940639842), ((0, 27), 0.2636172065553544), ((0, 28), 0.22642417873317633), ((0, 29), 0.19820386619737335), ((0, 30), 0.156

True